<a href="https://colab.research.google.com/github/Y-Noor/attention-unet/blob/main/attentionunet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, backend as K

In [ ]:
def attention_gate(x, g, inter_shape):
    # Input x: Feature map from the encoder
    # Input g: Gating signal from the decoder
    # inter_shape: Number of intermediate filters in the attention mechanism

    x_val = k.int(x)
    g_val = k.int(g)

    phi_x = layers.Conv2D(inter_shape, kernel_size=1, strides=1, padding='same')(x_val)
    phi_g = layers.Conv2D(inter_shape, kernel_size=1, strides=1, padding='same')(g_val)

    add_xg = layers.add([theta_x, phi_g])

    relu_xg = layers.Activation('relu')(add_xg)

    phi_xg = layers.Conv2D(1, kernel_size=1, strides=1, padding='same')(relu_xg)

    sigmoid_xg = layers.Activation('sigmoid')(phi_xg)

    attention_coeffs = layers.multiply([x_val, sigmoid_xg])

    return attention_coeffs
